##### Copyright 2019 Los autores de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Visualización de datos de imágenes en TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/image_summaries"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a></td>
</table>

## Visión general

Con la **API de resumen de imágenes de TensorFlow,** puede registrar fácilmente tensores e imágenes arbitrarias y verlas en TensorBoard. Esto puede resultar extremadamente útil para muestrear y examinar los datos de entrada o para [visualizar los pesos de las capas](http://cs231n.github.io/understanding-cnn/) y los[tensores generados](https://hub.packtpub.com/generative-adversarial-networks-using-keras/) . También puede registrar datos de diagnóstico como imágenes que pueden ser útiles en el curso del desarrollo de su modelo.

En este tutorial, aprenderá a utilizar la API de resumen de imágenes para visualizar tensores como imágenes. También aprenderá cómo tomar una imagen arbitraria, convertirla en un tensor y visualizarla en TensorBoard. Trabajará con un ejemplo simple pero real que utiliza resúmenes de imágenes para ayudarlo a comprender el rendimiento de su modelo.


## Configuración

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

TensorFlow 2.x selected.


In [4]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.2


# Descargue el conjunto de datos Fashion-MNIST

Vas a construir una red neuronal simple para clasificar imágenes en el conjunto de datos [Fashion-MNIST.](https://research.zalando.com/welcome/mission/research-projects/fashion-mnist/) Este conjunto de datos consta de 70.000 imágenes en escala de grises de 28x28 de productos de moda de 10 categorías, con 7.000 imágenes por categoría.

Primero, descargue los datos:

In [5]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

4423680/4422102 [==============================] - 0s 0us/step


## Visualizando una sola imagen

Para comprender cómo funciona la API de resumen de imágenes, ahora simplemente registrará la primera imagen de entrenamiento en su conjunto de entrenamiento en TensorBoard.

Antes de hacer eso, examine la forma de sus datos de entrenamiento:

In [6]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


Observe que la forma de cada imagen en el conjunto de datos es un tensor de forma de rango 2 (28, 28), que representa la altura y el ancho.

Sin embargo, `tf.summary.image()` espera un tensor de rango 4 que contenga `(batch_size, height, width, channels)` . Por lo tanto, es necesario remodelar los tensores.

Está registrando solo una imagen, por lo que `batch_size` es 1. Las imágenes son en escala de grises, así que configure los `channels` en 1.

In [ ]:
# Reshape the image for the Summary API.
img = np.reshape(train_images[0], (-1, 28, 28, 1))

Ahora está listo para registrar esta imagen y verla en TensorBoard.

In [ ]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

Ahora, use TensorBoard para examinar la imagen. Espere unos segundos a que la interfaz de usuario comience a funcionar.

In [ ]:
%tensorboard --logdir logs/train_data

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_single.png?raw=1"/> -->

La pestaña "Imágenes" muestra la imagen que acaba de iniciar sesión. Es una "bota de tobillo".

La imagen se escala a un tamaño predeterminado para facilitar la visualización. Si desea ver la imagen original sin escala, marque "Mostrar tamaño de imagen real" en la esquina superior izquierda.

Juega con los controles deslizantes de brillo y contraste para ver cómo afectan los píxeles de la imagen.

## Visualización de múltiples imágenes

Registrar un tensor es genial, pero ¿y si quisiera registrar varios ejemplos de entrenamiento?

Simplemente especifique la cantidad de imágenes que desea registrar al pasar datos a `tf.summary.image()` .

In [ ]:
with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_multiple.png?raw=1"/> -->

## Registro de datos de imágenes arbitrarios

¿Qué pasa si desea visualizar una imagen que no es un tensor, como una imagen generada por [matplotlib](https://matplotlib.org/) ?

Necesita un código repetitivo para convertir la trama en un tensor, pero después de eso, está listo para comenzar.

En el siguiente código, registrará las primeras 25 imágenes como una buena cuadrícula usando la función subplot `subplot()` matplotlib. Luego verá la cuadrícula en TensorBoard:

In [ ]:
# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
  
  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)

%tensorboard --logdir logs/plots

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_arbitrary.png?raw=1"/> -->

## Construyendo un clasificador de imágenes

Ahora pon todo esto junto con un ejemplo real. Después de todo, estás aquí para hacer aprendizaje automático y no para trazar imágenes bonitas.

Utilizará resúmenes de imágenes para comprender qué tan bien le está yendo a su modelo mientras entrena un clasificador simple para el conjunto de datos Fashion-MNIST.

Primero, cree un modelo muy simple y compílelo, configurando el optimizador y la función de pérdida. El paso de compilación también especifica que desea registrar la precisión del clasificador a lo largo del camino.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Al entrenar un clasificador, es útil ver la [matriz de confusión](https://en.wikipedia.org/wiki/Confusion_matrix) . La matriz de confusión le brinda un conocimiento detallado de cómo se está desempeñando su clasificador en los datos de prueba.

Defina una función que calcule la matriz de confusión. Utilizará una función conveniente de [Scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html) para hacer esto, y luego lo trazará usando matplotlib.

In [ ]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

Ahora está listo para entrenar al clasificador y registrar regularmente la matriz de confusión a lo largo del camino.

Esto es lo que harás:

1. Cree la [devolución de llamada de Keras TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) para registrar métricas básicas
2. Cree un [Keras LambdaCallback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LambdaCallback) para registrar la matriz de confusión al final de cada época
3. Entrene el modelo usando Model.fit (), asegurándose de pasar ambas devoluciones de llamada

A medida que avanza el entrenamiento, desplácese hacia abajo para ver cómo se inicia TensorBoard.

In [ ]:
# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Start TensorBoard.
%tensorboard --logdir logs/image

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_accuracy.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_cm.png?raw=1"/> -->

Tenga en cuenta que la precisión aumenta tanto en el tren como en los conjuntos de validación. Buena seńal. Pero, ¿cómo se está desempeñando el modelo en subconjuntos específicos de datos?

Seleccione la pestaña "Imágenes" para visualizar sus matrices de confusión registradas. Marque "Mostrar tamaño de imagen real" en la parte superior izquierda para ver la matriz de confusión a tamaño completo.

De forma predeterminada, el panel muestra el resumen de la imagen del último paso o época registrada. Utilice el control deslizante para ver matrices de confusión anteriores. Observe cómo la matriz cambia significativamente a medida que avanza el entrenamiento, los cuadrados más oscuros se fusionan a lo largo de la diagonal y el resto de la matriz tiende hacia el 0 y el blanco. ¡Esto significa que su clasificador está mejorando a medida que avanza el entrenamiento! ¡Buen trabajo!

La matriz de confusión muestra que este modelo simple tiene algunos problemas. A pesar del gran progreso, las camisas, camisetas y suéteres se están confundiendo entre sí. El modelo necesita más trabajo.

Si está interesado, intente mejorar este modelo con una [red convolucional](https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a) (CNN).